In [ ]:
import cv2
import mediapipe as mp
import pandas as pd
import os

# Initialize mediapipe pose and drawing utilities
mp_pose = mp.solutions.pose

# Crear una lista vacía para almacenar los datos
landmark_data = []

# Open the video file
cap = cv2.VideoCapture("IMG_0435.MOV")

# Initialize pose estimation (set `static_image_mode` to False for videos)
with mp_pose.Pose(static_image_mode=False) as pose:
    frame_count = 0
    while True:
        ret, frame = cap.read()  # Leer cada frame del video
        if not ret:
            break  # Salir del loop cuando no haya más frames

        # Convertir el frame a RGB para procesarlo
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(frame_rgb)  # Procesar los landmarks

        # Si se detectan landmarks, extraer las coordenadas
        if results.pose_landmarks:
            for id, lm in enumerate(results.pose_landmarks.landmark):
                # Añadir los datos a la lista
                landmark_data.append({
                    'frame': frame_count,
                    'landmark_id': id,
                    'x': lm.x,
                    'y': lm.y,
                    'z': lm.z,
                    'visibility': lm.visibility
                })

        frame_count += 1

# Liberar el video
cap.release()

# Convertir la lista en un dataframe
df = pd.DataFrame(landmark_data)

# Crear una carpeta para guardar el archivo CSV (si no existe ya)
output_folder = "output"
os.makedirs(output_folder, exist_ok=True)

# Guardar el dataframe en un archivo CSV dentro de la carpeta
csv_file_path = os.path.join(output_folder, 'tracked_landmarks.csv')
df.to_csv(csv_file_path, index=False)

print(f"Datos guardados en: {csv_file_path}")

In [ ]:
import pandas as pd
import os

# Cargar el CSV original con todos los landmarks trackeados
csv_file_path = 'output/tracked_landmarks.csv'  # Cambia esta ruta si es necesario
df = pd.read_csv(csv_file_path)

# Filtrar solo las filas donde landmark_id es 27
df_landmark_27 = df[df['landmark_id'] == 27]

# Crear una carpeta para guardar el nuevo archivo CSV (si no existe ya)
output_folder = "output"
os.makedirs(output_folder, exist_ok=True)

# Guardar el dataframe filtrado en un nuevo archivo CSV llamado 'trackeo_deseado.csv'
csv_file_path_landmark_27 = os.path.join(output_folder, 'trackeo_deseado.csv')
df_landmark_27.to_csv(csv_file_path_landmark_27, index=False)

print(f"Datos del landmark 27 guardados en: {csv_file_path_landmark_27}")


In [ ]:
def video():
  from google.colab import files
  uploaded = files.upload()
